# Import Libraries

In [1]:
from swarmintelligence import ParticleSwarmOptimization, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']

In [3]:
gray_misc_dataset

{'AirplaneF16.tiff': array([[ 65, 199, 193, ..., 166, 164, 148],
        [ 54, 196, 192, ..., 179, 155, 140],
        [ 54, 197, 194, ..., 190, 159, 141],
        ...,
        [ 67, 213, 213, ..., 184, 176, 151],
        [ 68, 214, 214, ..., 166, 184, 167],
        [ 89,  86,  88, ..., 112, 111, 107]], dtype=uint8),
 'Lena.png': array([[162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        ...,
        [ 43,  43,  50, ..., 104, 100,  98],
        [ 44,  44,  55, ..., 104, 105, 108],
        [ 44,  44,  55, ..., 104, 105, 108]], dtype=uint8),
 'Male.tiff': array([[130, 138, 140, ..., 115, 113, 111],
        [139, 134, 131, ..., 110, 117, 116],
        [141, 127, 131, ..., 109, 108, 107],
        ...,
        [102, 103, 103, ...,  76,  80,  89],
        [106, 102, 109, ...,  72,  69,  82],
        [ 98, 105, 116, ...,  85,  81,  89]], dtype=uint8),
 'Mandrill.tiff': array([[145,  56,  49, ..., 113, 153, 177],

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/PSO/optimal_params.pkl', 'rb'))
optimal_params

{'particleSize': 25,
 'maxIteration': 100,
 'phi1': 2.6,
 'phi2': 1.7,
 'inertia': 0.5}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['mMasi_entropy']
objs = ['max']
n_runs = 30

In [6]:
pso_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds':[],
    'Mean CPU_time (seconds)': [],
    'Mean Fitness':[],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions': [],
    'CPU_time (seconds)': [],
    'Fitness':[],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in gray_misc_dataset.items():
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                pso_evaluation_results['image_name'].append(image_name)
                pso_evaluation_results['thresholds'].append(threshold)
                pso_evaluation_results['fitness_function'].append(fitness_function)
                pso_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                fitness_values = []
                list_best_thresholds = []
                list_fitness = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    pso = ParticleSwarmOptimization(
                        k=threshold,
                        particleSize=optimal_params['particleSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        phi1=optimal_params['phi1'],
                        phi2=optimal_params['phi2'],
                        inertia=optimal_params['inertia']
                    )
                    # Evaluation Metrics
                    st = time.time()
                    particles, best_thresholds = pso.fit_run(image_array)
                    et = time.time()
                    regions = Utilization().digitize(image_array, best_thresholds)
                    list_cpu_time.append(et-st)
                    list_best_thresholds.append(best_thresholds)
                    fitness_values.append(pso.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, regions))
                    list_rmse.append(rmse(image_array, regions))
                    list_psnr.append(psnr(image_array, regions))
                    list_ssim.append(ssim(image_array, regions)[0])
                    list_uqi.append(uqi(image_array, regions))
                    print(pso.get_params_training_()['best_fitness_tracking'][-1])

                
                mean_best_thresholds = np.round(np.array(list_best_thresholds).mean(axis=0)).astype('int')
                mean_regions=Utilization().digitize(image_array, mean_best_thresholds)
                pso_evaluation_results['Mean best_thresholds'].append(mean_best_thresholds)
                pso_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                pso_evaluation_results['Mean Fitness'].append(np.mean(np.array(fitness_values)))
                pso_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                pso_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                pso_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                pso_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                pso_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                pso_evaluation_results['Regions'].append(mean_regions)
                pso_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                pso_evaluation_results['Fitness'].append(fitness_values)
                pso_evaluation_results['MSE'].append(list_mse)
                pso_evaluation_results['RMSE'].append(list_rmse)
                pso_evaluation_results['PSNR'].append(list_psnr)
                pso_evaluation_results['SSIM'].append(list_ssim)
                pso_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean(np.array(fitness_values))}")

	 AirplaneF16.tiff | Threshold =  2
1948.6765950132303
1948.6765950132303
1948.7106129145502
1948.7183902321062
1948.7106129145502
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.6717399702634
1948.7183902321062
1948.7183902321062
1948.6717399702634
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.6765950132303
1948.6765950132303
1948.7106129145502
1948.6765950132303
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7106129145502
1948.7183902321062
1948.7183902321062
1948.7106129145502
1948.7183902321062
1948.7183902321062
Mean Fitness = 1948.7070181252445
	 AirplaneF16.tiff | Threshold =  3
2024.8020017977378
2024.5704378674416
2024.5634930156143
2024.6943435813641
2024.6633145463807
2024.5864877130118
2024.5847077974663
2024.6120316280364
2024.5493869321924
2024.658116219674
2024.2953000685475
2024.8375098105803
2024.7401565921582
2024.8375098105803
2024.4913861988546
2024.6280179608698
2024.643913117201
202

3253.7248272461225
3250.109872120775
3210.7048062055637
3254.4757949601276
Mean Fitness = 3247.4039731877415
	 Mandrill.tiff | Threshold =  2
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1548.9859586850848
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1548.9952130399997
1548.9952130399997
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
Mean Fitness = 1549.0236323193035
	 Mandrill.tiff | Threshold =  3
1639.4542198585186
1639.1357358599644
1639.2256498527177
1639.2073269511038
1639.463360998692
1639.463360998692
1636.9188440585558
1639.466171099567
1638.9689179665552
1639.2073269511038
1639.466171099567
1639.38279183353

4217.969061608366
4217.856603559883
4217.972013024968
4217.93950735422
4217.759263271781
4217.914658720758
4217.958687837565
4217.902383477156
4217.969098793352
4217.759263271781
4217.926565484326
Mean Fitness = 4217.92580053584


## Visualize Results using DataFrame

In [7]:
pso_evaluation_results_df =pd.DataFrame(pso_evaluation_results)
pso_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds,Mean CPU_time (seconds),Mean Fitness,Mean MSE,Mean RMSE,Mean PSNR,Mean SSIM,Mean UQI,Regions,CPU_time (seconds),Fitness,MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,mMasi_entropy,max,"[113, 173]",1.810881,1948.707018,2267.483316,47.618066,14.575375,0.794478,0.940256,"[[113, 255, 255, 255, 255, 255, 255, 255, 255,...","[1.376833200454712, 1.1624009609222412, 1.3176...","[1948.6765950132303, 1948.6765950132303, 1948....","[2262.310806274414, 2262.310806274414, 2258.24...","[47.56375517423339, 47.56375517423339, 47.5210...","[14.585280908808967, 14.585280908808967, 14.59...","[0.7945291183281538, 0.7945291183281538, 0.794...","[0.9403035413955733, 0.9403035413955733, 0.940..."
1,AirplaneF16.tiff,3,mMasi_entropy,max,"[93, 145, 191]",1.891184,2024.607285,1766.626016,42.027665,15.660840,0.777508,0.953366,"[[93, 255, 255, 191, 191, 191, 255, 191, 255, ...","[1.8155486583709717, 1.4125454425811768, 1.395...","[2024.8020017977378, 2024.5704378674416, 2024....","[1779.3615493774414, 1703.5488357543945, 1804....","[42.18247917533346, 41.274069774549666, 42.481...","[15.62816159301072, 15.817257728350745, 15.566...","[0.7788206538325575, 0.7694642319532095, 0.783...","[0.953067156214341, 0.9546152402508755, 0.9525..."
2,AirplaneF16.tiff,4,mMasi_entropy,max,"[85, 132, 174, 204]",1.922654,2069.014804,1070.278164,32.689841,17.849375,0.733145,0.969388,"[[85, 204, 204, 204, 204, 204, 204, 204, 204, ...","[2.2026607990264893, 2.323207139968872, 2.1461...","[2069.099058290232, 2069.093293016048, 2069.26...","[1054.8141250610352, 977.9398536682129, 1092.1...","[32.477902103754104, 31.27202989363199, 33.047...","[17.89904424051604, 18.227682157155375, 17.748...","[0.7308860091690064, 0.7347343285580662, 0.730...","[0.9691837049973008, 0.971300866700407, 0.9698..."
3,AirplaneF16.tiff,5,mMasi_entropy,max,"[76, 114, 149, 184, 206]",1.682098,2093.320384,847.531314,29.066342,18.876725,0.746641,0.976042,"[[76, 206, 206, 206, 184, 206, 206, 206, 206, ...","[1.8371596336364746, 1.6164493560791016, 1.785...","[2093.9811288389983, 2094.4552098713193, 2093....","[817.3686103820801, 800.6012802124023, 848.743...","[28.589659151205005, 28.294898483868117, 29.13...","[19.00662405495286, 19.09664080633751, 18.8430...","[0.7448170563747303, 0.7485547689947767, 0.750...","[0.9763960774368621, 0.976872336774735, 0.9761..."
4,Lena.png,2,mMasi_entropy,max,"[93, 151]",1.722737,1961.738300,2794.017951,52.856108,13.669290,0.612937,0.883749,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...","[1.6689045429229736, 1.6714136600494385, 1.692...","[1961.8220125153634, 1961.8220125153634, 1961....","[2813.567070007324, 2813.567070007324, 2812.93...","[53.04306806744237, 53.04306806744237, 53.0371...","[13.638230885124642, 13.638230885124642, 13.63...","[0.6121817460412936, 0.6121817460412936, 0.614...","[0.8833090574580458, 0.8833090574580458, 0.883..."
5,Lena.png,3,mMasi_entropy,max,"[81, 127, 171]",1.747871,2127.347894,1251.570023,35.372314,17.158816,0.705961,0.933287,"[[171, 171, 171, 171, 171, 171, 171, 171, 171,...","[1.2908647060394287, 1.5408642292022705, 1.725...","[2128.123585004371, 2127.8276960185262, 2127.5...","[1227.5580368041992, 1213.2176322937012, 1309....","[35.036524325397906, 34.83127376788999, 36.185...","[17.240383269014373, 17.29141647366268, 16.960...","[0.7058130850805338, 0.7071698327151489, 0.706...","[0.9335951769584808, 0.9343860059521882, 0.933..."
6,Lena.png,4,mMasi_entropy,max,"[77, 115, 147, 182]",1.861865,2188.137639,804.852746,28.359355,19.080061,0.752149,0.951912,"[[182, 182, 182, 182, 182, 182, 182, 182, 182,...","[1.734342336654663, 1.3862700462341309, 1.2810...","[2188.1785180939396, 2184.964381468718, 2190.3...","[859.4664154052734, 820.8205642700195, 775.244...","[29.316657643825522, 28.649966217606952, 27.84...","[18.78851450047442, 18.98832132450659, 19.2364...","[0.7494114824014804, 0.7481954702887823, 0.757...","[0.9535124729075388, 0.9478851934837522, 0.951..."
7,Lena

# Save results

In [8]:
pickle.dump(pso_evaluation_results_df, open('results/evaluation/PSO/PSO_masi_df.pkl', 'wb'))
pickle.dump(pso_evaluation_results, open('results/evaluation/PSO/PSO_masi_dict.pkl', 'wb'))